<a href="https://colab.research.google.com/github/yampaidelimon/e1-frontend/blob/main/Ejercicios_taller_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*texto en cursiva*#Guía: SQL

En esta guía usted deberá realizar consultas SQL en un servidor virtual con PostgreSQL que contiene
datos de películas extraídos de IMDb. El esquema de los datos es el siguiente:

* $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\underline{\text{anho}}}, \color{blue}{\text{calificacion}})$
* $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\text{genero}})$
* $\color{green}{\textbf{personaje}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}},  \color{blue}{\underline{\text{a_nombre}}}, \color{blue}{\underline{\text{personaje}}})$

La tabla $\color{green}{\textbf{personaje}}$ usa llaves foráneas que hacen referencia a las tablas de $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{a_nombre}}})$ y $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}})$.

Para iniciar el servidor virtual, instalar la base de datos postgres, y descargar los datos e importarlos, debe correr el siguiente bloque:



In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
!wget -cO - https://drive.google.com/uc\?export\=download\&id\=1g7UsolqIt5CatIUiWKQNiQ4sT6zsCZnk > /content/guia2.sql
!psql postgres < /content/guia2.sql
%sql set search_path to guia;

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [55.4 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [929 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu 

""


Ejecute la siguiente consulta para probar que todo ande bien:

In [ ]:
%sql select * from pelicula;

 * postgresql+psycopg2://@/postgres


,nombre,anho,calificacion
0,The Shawshank Redemption,1994,9.2
1,The Godfather,1972,9.2
2,The Godfather: Part II,1974,9.0
3,The Dark Knight,2008,8.9
4,12 Angry Men,1957,8.9
...,...,...,...
245,The King's Speech,2010,8.0
246,The Avengers,2012,8.0
247,Lagaan: Once Upon a Time in India,2001,8.0
248,Beauty and the Beast,1991,8.0


Para ejecutar consultas multi-lineas use el tag %%sql:

In [ ]:
%%sql
SELECT * FROM
pelicula
where nombre ilike '%terminator%';

 * postgresql+psycopg2://@/postgres


,nombre,anho,calificacion
0,Terminator 2: Judgment Day,1991,8.5
1,The Terminator,1984,8.0


Ahora, debe diseñar las consultas que resuelvan las siguientes preguntas usando los operadores vistos en clases.

#SQL
## Pregunta 1
 Crear un Trigger para Actualizar la Calificación Promedio de una Película

In [ ]:
%sql drop MATERIALIZED view CalificacionPromedio

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.UndefinedTable) materialized view "calificacionpromedio" does not exist

[SQL: drop MATERIALIZED view CalificacionPromedio]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [ ]:
%%sql
CREATE MATERIALIZED VIEW CalificacionPromedio AS
SELECT nombre, AVG(calificacion) AS promedio
FROM pelicula
GROUP BY nombre

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql select  * from CalificacionPromedio

 * postgresql+psycopg2://@/postgres


,nombre,promedio
0,L.A. Confidential,8.3
1,Before Sunrise,8.0
2,Million Dollar Baby,8.1
3,Platoon,8.1
4,Rashômon,8.3
...,...,...
245,Batman Begins,8.2
246,The Martian,8.0
247,Catch Me If You Can,8.0
248,All About Eve,8.2


In [ ]:
%%sql
-- Step 1: Create the function to refresh the materialized view
CREATE OR REPLACE FUNCTION actualizar_calificacion_promedio()
RETURNS TRIGGER AS $$
BEGIN
  REFRESH MATERIALIZED VIEW CalificacionPromedio;
  RETURN NULL;
END;
$$ LANGUAGE plpgsql;

-- Step 2: Create the trigger that calls the function after an INSERT or UPDATE
CREATE TRIGGER actualizar_calificacion_promedio
AFTER INSERT OR UPDATE ON pelicula
FOR EACH STATEMENT
EXECUTE FUNCTION actualizar_calificacion_promedio();

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql insert into pelicula (nombre, anho, calificacion) Values(L.A. Confidential	8.3);

In [ ]:
%sql select  * from CalificacionPromedio

## Pregunta 2
Crear un Trigger para Evitar Duplicados de Actores en la Misma Película

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION evitar_actor_duplicado()
RETURNS TRIGGER AS $$
BEGIN
  IF EXISTS (SELECT 1 FROM actor WHERE nombre = NEW.nombre) THEN
    RAISE EXCEPTION 'Actor % ya existe.', NEW.nombre;
  END IF;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER evitar_actor_duplicado_trigger
BEFORE INSERT ON actor
FOR EACH ROW
EXECUTE FUNCTION evitar_actor_duplicado();

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
SELECT
    pelicula.nombre AS pelicula_nombre,
    pelicula.anho AS pelicula_anho,
    pelicula.calificacion AS pelicula_calificacion,
    actor.nombre AS actor_nombre,
    actor.genero AS actor_genero,
    personaje.personaje AS personaje_nombre
FROM
    pelicula
JOIN
    personaje ON pelicula.nombre = personaje.p_nombre AND pelicula.anho = personaje.p_anho
JOIN
    actor ON personaje.a_nombre = actor.nombre;

 * postgresql+psycopg2://@/postgres


,pelicula_nombre,pelicula_anho,pelicula_calificacion,actor_nombre,actor_genero,personaje_nombre
0,Scarface,1983,8.3,"Aames, Angela",F,Woman at Babylon Club
1,The Best Years of Our Lives,1946,8.0,"Aames, Marlene",F,Luella Parrish
2,Amores perros,2000,8.1,"Abad, Regina",F,Jimena
3,Taxi Driver,1976,8.3,"Abbott, Diahnne",F,Concession Girl
4,The Apartment,1960,8.3,"Abbott, Dorothy",F,Office Worker
...,...,...,...,...,...,...
17224,Hauru no ugoku shiro,2004,8.2,"Ôtsuka, Akio",M,Kokuô
17225,Shichinin no samurai,1954,8.6,"Ôtsuka, Hideo",M,Farmer
17226,Kaze no tani no Naushika,1984,8.1,"Ôtsuka, Hôchû",M,Torumekia Soldier
17227,Tenkû no shiro Rapyuta,1986,8.0,"Ôtsuka, Hôchû",M,Additional Voices


In [ ]:
%%sql
INSERT INTO actor VALUES('Aames, Marlene', 'F')

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.RaiseException) Actor Aames, Marlene ya existe.
CONTEXT:  PL/pgSQL function evitar_actor_duplicado() line 4 at RAISE

[SQL: INSERT INTO actor VALUES('Aames, Marlene', 'F')]
(Background on this error at: https://sqlalche.me/e/20/2j85)


## Pregunta 3
Función para Obtener la Filmografía de un Actor

In [ ]:
%%sql

DROP FUNCTION IF EXISTS obtener_filmografia(TEXT)

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
CREATE OR REPLACE FUNCTION obtener_filmografia(actor_nombre TEXT)
RETURNS TABLE (
    pelicula_nombre VARCHAR,
    pelicula_anho SMALLINT,
    personaje_nombre VARCHAR
) AS $$
BEGIN
    RETURN QUERY
    SELECT
        pelicula.nombre AS pelicula_nombre,
        pelicula.anho AS pelicula_anho,
        personaje.personaje AS personaje_nombre
    FROM
        pelicula
    JOIN
        personaje ON pelicula.nombre = personaje.p_nombre AND pelicula.anho = personaje.p_anho
    JOIN
        actor ON personaje.a_nombre = actor.nombre
    WHERE
        actor.nombre = actor_nombre;
END;
$$ LANGUAGE plpgsql


 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
SELECT * FROM obtener_filmografia('Damon, Matt')


 * postgresql+psycopg2://@/postgres


,pelicula_nombre,pelicula_anho,personaje_nombre
0,Good Will Hunting,1997,Will Hunting
1,Interstellar,2014,Mann
2,Saving Private Ryan,1998,Private Ryan
3,The Bourne Ultimatum,2007,Jason Bourne
4,The Departed,2006,Colin Sullivan
5,The Martian,2015,Mark Watney


## Pregunta 4
Consulta para Obtener el Número de Actores por Género

In [ ]:
%%sql
select genero, COUNT(genero)
from actor
group by genero

 * postgresql+psycopg2://@/postgres


,genero,count
0,M,10928
1,F,3748


## Pregunta 5
Películas con Más de Tres Actores Principales

In [ ]:
%%sql
SELECT pelicula.nombre, COUNT(DISTINCT actor.nombre) AS NumeroActoresPrincipales
FROM pelicula
JOIN personaje ON pelicula.nombre = personaje.p_nombre AND pelicula.anho = personaje.p_anho
JOIN actor ON personaje.a_nombre = actor.nombre
GROUP BY pelicula.nombre
HAVING COUNT(DISTINCT actor.nombre) > 3
ORDER BY NumeroActoresPrincipales ASC;

 * postgresql+psycopg2://@/postgres


,nombre,numeroactoresprincipales
0,Persona,5
1,Rashômon,8
2,Alien,10
3,Stalker,10
4,La battaglia di Algeri,11
...,...,...
245,Mr. Smith Goes to Washington,212
246,The Dark Knight,232
247,Citizen Kane,247
248,The Wolf of Wall Street,331


## Pregunta 6
Crea un trigger que registre en una tabla audit_actor cada vez que se inserte un nuevo registro en la tabla actor.

In [ ]:
%%sql

DROP FUNCTION IF EXISTS registrar_actor()

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.DependentObjectsStillExist) cannot drop function registrar_actor() because other objects depend on it
DETAIL:  trigger registrar_actor_trigger on table actor depends on function registrar_actor()
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: DROP FUNCTION IF EXISTS registrar_actor()]
(Background on this error at: https://sqlalche.me/e/20/2j85)


In [ ]:
%%sql
CREATE OR REPLACE FUNCTION registrar_actor()
RETURNS TRIGGER AS $$
BEGIN
  CREATE TABLE IF NOT EXISTS audit_actor(id SERIAL PRIMARY KEY, fecha TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP, actor TEXT NOT NULL);
  INSERT INTO audit_actor (actor) VALUES (NEW.nombre);
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER registrar_actor_trigger
BEFORE INSERT ON actor
FOR EACH ROW
EXECUTE FUNCTION registrar_actor();

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
INSERT INTO actor VALUES ('Arenas, Dante', 'M')

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM audit_actor

 * postgresql+psycopg2://@/postgres


,id,fecha,actor
0,1,2024-06-19 03:53:28.602765,"Arenas, Dante"


## Pregunta 7
Crear una Vista para Películas y Actores

In [ ]:
%%sql
CREATE OR REPLACE VIEW PeliculasActores
AS
SELECT pelicula.nombre as Pelicula, actor.nombre as Actor, personaje.personaje as Personaje
FROM pelicula, actor, personaje
WHERE personaje.p_nombre = pelicula.nombre
AND personaje.p_anho = pelicula.anho
AND personaje.a_nombre = actor.nombre
ORDER BY Pelicula ASC

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM PeliculasActores

 * postgresql+psycopg2://@/postgres


,pelicula,actor,personaje
0,12 Angry Men,"Balsam, Martin",Juror 1
1,12 Angry Men,"Webber, Robert (I)",Juror 12
2,12 Angry Men,"Gorman, Tom (I)",Stenographer
3,12 Angry Men,"Nelson, Billy (I)",Court Clerk
4,12 Angry Men,"Marshall, E.G.",Juror 4
...,...,...,...
17224,Zootopia,"Howard, Byron (I)",Bucky Oryx-Antlerson
17225,Zootopia,"Persi, Raymond S.",Flash
17226,Zootopia,"Bell, Kristen (I)",Priscilla
17227,Zootopia,"Goodwin, Ginnifer",Judy Hopps


## Pregunta 8
Crear una Vista para el Promedio de Calificaciones por Género de Actor

In [ ]:
%%sql
CREATE OR REPLACE VIEW CalificacionesPorGenero
AS
SELECT actor.genero, AVG(pelicula.calificacion)
FROM pelicula, actor, personaje
WHERE personaje.p_nombre = pelicula.nombre
AND personaje.p_anho = pelicula.anho
AND personaje.a_nombre = actor.nombre
GROUP BY actor.genero

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
SELECT *
FROM CalificacionesPorGenero

 * postgresql+psycopg2://@/postgres


,genero,avg
0,M,8.287732
1,F,8.251795


## Pregunta 9
Crear una Vista para Películas y sus Actores Principales

In [ ]:
???